# Acompanhamento - Atividades Jira

## GRUPO SEI

---

### Informações iniciais

**Edite apenas aqui!**

In [14]:
# --------------------------------------------------
# SPRINT A ANALISAR
# --------------------------------------------------
SPRINT = "07"


### Bibliotecas

In [15]:
from datetime import date, datetime, timedelta
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import requests
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Leitura do CSV

In [16]:
# --------------------------------------------------
# DATAS
# --------------------------------------------------
sprints = {
    "01":{"startday":"10-01-2023", "endday":"27-01-2023"},
    "02":{"startday":"30-01-2023", "endday":"17-02-2023"},
    "03":{"startday":"20-02-2023", "endday":"10-03-2023"},
    "04":{"startday":"13-03-2023", "endday":"31-03-2023"},
    "05":{"startday":"03-04-2023", "endday":"21-04-2023"},
    "06":{"startday":"24-04-2023", "endday":"12-05-2023"},
    "07":{"startday":"15-05-2023", "endday":"02-06-2023"},
    "08":{"startday":"05-06-2023", "endday":"23-06-2023"},
    "09":{"startday":"--2023", "endday":"--2023"},
    "10":{"startday":"--2023", "endday":"--2023"},
    "11":{"startday":"--2023", "endday":"--2023"},
    "12":{"startday":"--2023", "endday":"--2023"},
    "13":{"startday":"--2023", "endday":"--2023"},
    "14":{"startday":"--2023", "endday":"--2023"},
    "15":{"startday":"--2023", "endday":"--2023"},
    "16":{"startday":"--2023", "endday":"--2023"},
    "17":{"startday":"--2023", "endday":"--2023"},
}

HOJE = date.today().strftime("%d-%m-%Y")   
DATA_INICIAL = sprints[SPRINT]["startday"]
DATA_FINAL = sprints[SPRINT]["endday"]
startday = datetime.strptime(DATA_INICIAL, "%d-%m-%Y")
endday = datetime.strptime(DATA_FINAL, "%d-%m-%Y")
today = datetime.strptime(HOJE, "%d-%m-%Y")


# --------------------------------------------------
# Leitura do CSV
# --------------------------------------------------
if (today >= endday):
    linhas = open("Sprint{}/csv/Sprint_{}_{}.csv".format(SPRINT, SPRINT, DATA_FINAL),'r').read()
else:
    linhas = open("Sprint{}/csv/Sprint_{}_{}.csv".format(SPRINT, SPRINT, HOJE),'r').read()

fields = (linhas.split("Status Category\n")[0]+"Status Category").split(',')

linhas = linhas.split("Status Category\n")[1]
linhas = linhas.replace("Done\n","Done\n\r\r\r").replace("In Progress\n","In Progress\n\r\r\r").replace("To Do\n","To Do\n\r\r\r")
linhas = linhas.replace(", "," ")
linhas = linhas.replace(",height",";")
linhas = linhas.replace("a,NB4","a;NB4")
atividade = linhas.split("\n\r\r\r")[:-1]
# --------------------------------------------------

### Tratamento dos dados e informações a coletar

In [19]:
# --------------------------------------------------
# Variáveis a analisar
# --------------------------------------------------
projetos = {}

status = {
            "Done":{"pontos":0, "natividades":0},
            "In Progress":{"pontos":0, "natividades":0},
            "To Do":{"pontos":0, "natividades":0}
         }

scrum_days = {}
for i in range(15):
    scrum_days[startday + timedelta(days=(7*int(i/5)+(i%5)))] = {"nday":i+1,
                                                                  "points":0,
                                                                  "integrated points":0}

# --------------------------------------------------
# TRATAMENTO DOS DADOS - POR ATIVIDADE (Cada atividade é um item da lista)
# --------------------------------------------------
for i in range(len(atividade)):
    print(atividade[i].split(","))
    activityProject = atividade[i].split(",")[fields.index("Project key")]
    activityPoint = float(atividade[i].split(",")[fields.index("Custom field (Story Points)")])
    activityStatus = atividade[i].split(",")[fields.index("Status Category")]
    activityResolved = atividade[i].split(",")[fields.index("Resolved")]
    
    if activityProject not in projetos.keys():
        projetos[activityProject] = {"name":atividade[i].split(",")[fields.index("Project name")],
                         "totalPoints":0,
                         "dayInfo":{},
                         "status":{"Done":{"pontos":0, "natividades":0},
                                    "In Progress":{"pontos":0, "natividades":0},
                                    "To Do":{"pontos":0, "natividades":0}}
                                    }
        for i in range(15):
            projetos[activityProject]["dayInfo"][startday + timedelta(days=(7*int(i/5)+(i%5)))] = {"nday":i+1,
                                                                  "points":0,
                                                                  "integrated points":0}

    
    
    projetos[activityProject]["totalPoints"] += activityPoint
    
  

    for stat in status.keys():
        if (stat in activityStatus):
            projetos[activityProject]["status"][stat]['pontos'] += activityPoint
            projetos[activityProject]["status"][stat]['natividades'] += 1

     
    if activityResolved != "":
        resolved = datetime.strptime(activityResolved.split(" ")[0], "%d/%b/%y")
        try:
            projetos[activityProject]["dayInfo"][resolved]["points"] += activityPoint
        except:
            continue
# --------------------------------------------------



# --------------------------------------------------
# PONTOS INTEGRADOS - BURNUP
# --------------------------------------------------

for proj in projetos.keys():
    for day in projetos[proj]["dayInfo"].keys():
        if (day == startday):
            projetos[proj]["dayInfo"][day]["integrated points"] = projetos[proj]["dayInfo"][day]["points"]
        else:
            previous_day = list(projetos[proj]["dayInfo"].keys())[projetos[proj]["dayInfo"][day]["nday"]-2]
            projetos[proj]["dayInfo"][day]["integrated points"] = projetos[proj]["dayInfo"][day]["points"] + projetos[proj]["dayInfo"][previous_day]["integrated points"]
# --------------------------------------------------


['Imprimir cases para sensores de vazamento', 'PENT004012-21', '30167', 'Story', 'To Do', 'PENT004012', 'ENT004.012 - MELHORIAS E CARACTERIZAÇÃO DAS FONTES DO SIRIUS', 'software', 'Sergio Rodrigo Marques', '6266f26ca32183006f233fb7', '"ESCOPO:\nMelhorias necessárias para as fontes do Sirius observadas ao longo do tempo seja devido à falta de tempo para o término dos projetos seja por observações pós instalação a saber: \n1) a campanha de caracterização de ripple das fontes focando nas interferências de 2 Hz e 60 Hz.\n2) a ativação da proteção contra corrente de fuga para o terra (desativada no momento da criação deste documento). \n3) introduzir detecção e proteção contra vazamentos nos racksdas fontes refrigeradas. \n4) a solução do problema de sobreaquecimento das fontes Regratron."', '', 'High', '', 'Patricia Henriques Nallin', '62b0acb8f38b4dcf73d839ec', 'Patricia Henriques Nallin', '62b0acb8f38b4dcf73d839ec', 'Patricia Henriques Nallin', '62b0acb8f38b4dcf73d839ec', '09/May/23 2:26

ValueError: could not convert string to float: '2023 Sprint 7'

### Plots

In [18]:
# --------------------------------------------------
# PLOTS
# --------------------------------------------------
plot_data = {"date":[]}
for day in scrum_days.keys():
    plot_data["date"].append(day)

for proj in projetos:
    plot_data[proj] = {"burnup":[], "burndown":[], "daily":[]}
    for day in scrum_days.keys():
        plot_data[proj]["burnup"].append(projetos[proj]["dayInfo"][day]["integrated points"])
        plot_data[proj]["burndown"].append(int(projetos[proj]["totalPoints"] - projetos[proj]["dayInfo"][day]["integrated points"]))
        plot_data[proj]["daily"].append(int(projetos[proj]["dayInfo"][day]["points"]))

    
    
    
# --------------------------------------------------
# BURNDOWN
# --------------------------------------------------  
fig = go.Figure()

fig = make_subplots(rows=len(projetos.keys()), cols=1,
                    shared_xaxes=False,
                    vertical_spacing=0.03,
                    subplot_titles = list(projetos.keys()))

for i, proj in enumerate(projetos.keys()):
    fig.add_trace(go.Scatter(x=[startday, endday], y=[projetos[proj]["totalPoints"], 0],
                        mode='lines',
                        name='ideal'),
                        row=i+1, col=1)
    fig.add_trace(go.Scatter(x=plot_data["date"], y=plot_data[proj]["burndown"],
                        mode='lines+markers',
                        name='Pontos a concluir'),
                        row=i+1, col=1)



fig.update_layout(
    title={
        'text': "Burndown POR PROJETO - Grupo SEI - Sprint {}/{}     -     ({})\n".format(SPRINT, HOJE[-4:], HOJE),
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    yaxis_title="Pontos",
    showlegend=False,
    height=3000,
    width=750
)
fig.show()
plotly.offline.plot(fig, filename = 'Sprint{}/burndown_PROJETOS_SEI_Sprint_{}_{}.html'.format(SPRINT, SPRINT, HOJE[-4:]), auto_open=False)



'Sprint07/burndown_PROJETOS_SEI_Sprint_07_2023.html'